In [1]:
from kokoro import KModel
import torch
from misaki import en
import numpy as np
import os
import IPython.display as ipd


/home/azureuser/miniconda3/envs/kokoro_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_bin_voice(voice_path: str) -> torch.Tensor:
    """
    Load a .bin voice file as a PyTorch tensor.
    
    Args:
        voice_path: Path to the .bin voice file
        
    Returns:
        PyTorch tensor containing the voice data
    """
    if not os.path.exists(voice_path):
        raise FileNotFoundError(f"Voice file not found: {voice_path}")
    
    if not voice_path.endswith('.bin'):
        raise ValueError(f"Expected a .bin file, got: {voice_path}")
    
    # Load the binary file as a numpy array of float32 values
    voice_data = np.fromfile(voice_path, dtype=np.float32).reshape(-1, 1, 256)
    # Convert to PyTorch tensor
    voice_tensor = torch.tensor(voice_data, dtype=torch.float32)
    
    # Return the tensor
    return voice_tensor

In [59]:
ref_s.shape

torch.Size([1, 256])

In [5]:
# Initialize pipeline with American English
g2p = en.G2P(trf=False, british=False, fallback=None, unk='')

# Example 1: Using the pipeline with a voice name
text = "This is a test! I'm going to the store."
ps, mtoks = g2p(text)
model = KModel(repo_id="hexgrad/Kokoro-82M").to("cpu").eval()
tokens = list(filter(lambda i: i is not None, map(lambda p: model.vocab.get(p), ps)))

ref_s = load_bin_voice("kokoro.js/voices/af_heart.bin")[len(tokens)-1]
input_ids = torch.tensor([[0,*tokens,0], [0,*tokens[:30],0]], dtype=torch.long)
audio, _ = model.forward_with_tokens(input_ids, ref_s, 1.0)
ipd.Audio(audio, rate=24000)

ValueError: expected sequence of length 41 at dim 1 (got 32)

In [20]:
a= torch.arange(10)
dur_a = torch.ones(10, dtype=torch.long)
dur_a[3:7] = 2

a= a.unsqueeze(0)

indices = torch.repeat_interleave(torch.arange(a.shape[1]), dur_a)
pred_aln_trg = torch.zeros((a.shape[1], a.shape[0]))
pred_aln_trg[indices, torch.arange(a.shape[0])] = 1
pred_aln_trg = pred_aln_trg.unsqueeze(0)
print(pred_aln_trg.shape, pred_aln_trg)


torch.Size([1, 10, 1]) tensor([[[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]])


In [49]:
a= torch.ones(1024, 5000, 1).transpose(0,1)
conv = torch.nn.utils.weight_norm(torch.nn.Conv1d(1, 1, kernel_size=3, stride=2, groups=1, padding=1))
torch.cat([a, conv(torch.ones(5000,1).unsqueeze(1)),conv(torch.ones(5000,1).unsqueeze(1))],axis=1).shape

torch.Size([5000, 1026, 1])

In [57]:
torch.ones(5,1,5000)+torch.nn.functional.interpolate(torch.ones(5,1,5000), scale_factor=2, mode="nearest")

RuntimeError: The size of tensor a (5000) must match the size of tensor b (10000) at non-singleton dimension 2